In [1]:
import cv2
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
## TrainData 
train_json_path = './dataset/train/train_defected_json'
train_img_path = './dataset/train/train_defected_dataset'
json_files = []
img_files = []

for idx in os.listdir(train_json_path):
    # 확장자 명 제거
    idx = idx[:-5]

    # json files load
    with open(f'{train_json_path}/{idx}.json', 'r') as f:
        json_files.append(json.load(f))
    
    # image files load
    img_files.append(cv2.imread(f'{train_img_path}/{idx}.png', cv2.IMREAD_GRAYSCALE))

In [3]:
def make_rectangle(img: np.array, js: dict) -> np.array:
    # 사각형 좌표 구하기
    top,bot = (js['top_x'],js['top_y']),(js['bot_x'],js['bot_y'])

    # 사각형 그리기
    cv2.rectangle(img,top,bot,(255,255,255)) # Grayscale 이라서 흰색으로 그려야됨
    
    return img

def otsu_algorithm(img: np.array) -> np.array:
    ret, otsu = cv2.threshold(img,-1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return otsu

In [5]:
# 파일 경로 만들기
if not os.path.isdir('./dataset/Bboximages/train'):
    os.makedirs('./dataset/Bboximages/train/st')
    os.makedirs('./dataset/Bboximages/train/aq')
    os.makedirs('./dataset/Bboximages/train/fl')

# 오츠 알고리즘 적용
img_files = list(map(otsu_algorithm,img_files))

# 결함위치에 Bbox 그리기
for i in range(len(json_files)):
    temp = make_rectangle(img_files[i],json_files[i])
    # 결함 종류에 따라 다르게 저장하기
    cv2.imwrite(f'./dataset/Bboximages/train/{json_files[i]["defect_class"]}/{json_files[i]["image_name"]}',temp)